# Import

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
from sqlalchemy import create_engine
import pandas as pd
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import datetime

In [3]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.style.use('fivethirtyeight')

# Calculate Status for all the patients that has responses

### Query data

In [17]:
df_patient_response = pd.read_csv('../../data/patient_response.csv')
df_patient = pd.read_csv('../../data/patient_detail.csv')
df_tmetryentry = pd.read_csv('../../data/tmetryentry.csv')

### How each dataframe looks like

In [18]:
#Information for all of the entries
df_tmetryentry.head()

,id,session_id,date,object,action,strValue,unit_id,user_detail_id,intValue,just_date
28620,29918,2139699710,2017-05-15 13:07:14.407210,1,201,nm177724,3.0,67,NaN,2017-05-15
28621,29919,2139699710,2017-05-15 13:07:14.536299,1,203,nm177724,3.0,67,NaN,2017-05-15
28622,29920,2139699710,2017-05-15 13:07:14.536680,2,204,nm177724,3.0,67,NaN,2017-05-15
28623,29921,2139699710,2017-05-15 13:07:14.567145,2,301,nm177724,3.0,67,NaN,2017-05-15
28624,29922,2139699710,2017-05-15 13:07:14.567565,3,302,nm177724,3.0,67,NaN,2017-05-15


In [40]:
df_tmetryentry[df_tmetryentry.strValue == '098d477fde14ee3ee30c9f363d5ac06f85c3ca51']

,id,session_id,date,object,action,strValue,unit_id,user_detail_id,intValue,just_date
28643,29941,2139699710,2017-05-15 13:09:37.229240,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
28680,29978,2139699710,2017-05-15 13:12:51.209708,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
28690,29988,2139699710,2017-05-15 13:16:12.955344,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
28775,30073,3547602005,2017-05-15 13:20:51.575758,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
29154,30452,172184393,2017-05-15 14:33:32.844021,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-15
29389,30687,334320999,2017-05-16 14:47:00.542595,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-16
251968,253266,3860602040,2017-05-17 14:46:13.181498,6,602,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,3.0,67,NaN,2017-05-17


In [39]:
#information on the answers
df_patient_response[df_patient_response.patient_id.isin([120])]

,id,checklist_id,question_id,workday,response,date,patient_id,unit_id,user_detail_id,hash_fin,just_date
1741,1783,2,$0401,2017-05-31,A,2017-05-31 14:42:59.749305,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
1742,1784,2,$0402,2017-05-31,Y,2017-05-31 14:43:00.902044,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
1743,1785,2,$0501,2017-05-31,N,2017-05-31 14:43:02.948959,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
1744,1786,2,$0701,2017-05-31,N,2017-05-31 14:43:06.083190,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
1745,1787,2,$0702,2017-05-31,Y,2017-05-31 14:43:07.549812,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
1746,1788,2,$0703,2017-05-31,N,2017-05-31 14:43:08.783249,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
1747,1789,2,$0801,2017-05-31,Y,2017-05-31 14:43:11.433846,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
1748,1790,2,$0301,2017-05-31,N,2017-05-31 14:43:39.367822,120,4,71,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-31
1846,1888,2,$0801,2017-06-01,Y,2017-06-01 14:32:14.329502,120,4,75,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-06-01
1847,1889,2,$0401,2017-06-01,N,2017-06-01 14:35:19.665822,120,4,75,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-06-01


In [20]:
#information on the patients
df_patient.head()

,id,hash_fin,date,checklist_id,visible_groups,los_icu,los,apache_is_completed,has_antibiotics,has_lines,has_medication,has_ventilation,workday,unit_id,user_detail_id,just_date
44,120,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-15 13:09:39.642624,1,"08,07,03,10",3 days,9 days,0,0,1,1,0,2017-05-15,3,67,2017-05-15
45,121,37e2ca907b6b9c2fb5202767c096a73b6c18c5d9,2017-05-15 13:09:41.830361,1,"01,03,06,07,08",42 days,42 days,1,1,1,1,1,2017-05-15,4,69,2017-05-15
46,122,098d477fde14ee3ee30c9f363d5ac06f85c3ca51,2017-05-15 13:12:53.666151,1,"03,07,08,10",3 days,9 days,0,0,1,1,0,2017-05-15,3,67,2017-05-15
47,123,37e2ca907b6b9c2fb5202767c096a73b6c18c5d9,2017-05-15 13:13:53.532611,1,"01,06,08,03,07",42 days,42 days,1,1,1,1,1,2017-05-15,4,69,2017-05-15
48,124,37e2ca907b6b9c2fb5202767c096a73b6c18c5d9,2017-05-15 13:14:22.852311,1,"01,03,06,07,08",42 days,42 days,1,1,1,1,1,2017-05-15,4,69,2017-05-15


In [74]:
df_patient[df_patient.hash_fin == '07e1b75fe53343221703226deb96b71bb6d90930']

,id,hash_fin,date,checklist_id,visible_groups,los_icu,los,apache_is_completed,has_antibiotics,has_lines,has_medication,has_ventilation,workday,unit_id,user_detail_id,just_date
64,140,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-15 15:37:12.688794,1,"01,06,03,08,07",18 days,26 days,1,1,1,1,1,2017-05-15,4,69,2017-05-15
70,146,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-16 14:57:27.915578,1,"01,06,08,03,07",19 days,27 days,1,1,1,1,1,2017-05-16,4,69,2017-05-16
87,163,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-17 14:42:12.291822,1,"01,07,06,08,03",20 days,28 days,1,1,1,1,1,2017-05-17,4,69,2017-05-17
131,207,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-18 15:07:32.821076,1,"01,07,06,03,08",21 days,29 days,1,1,1,1,1,2017-05-18,4,72,2017-05-18
153,229,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-19 15:17:01.053216,1,"01,03,08,07,06",22 days,30 days,1,1,1,1,1,2017-05-19,4,71,2017-05-19
154,230,07e1b75fe53343221703226deb96b71bb6d90930,2017-05-19 15:19:09.764039,1,"01,03,08,07,06",22 days,30 days,1,1,1,1,1,2017-05-19,4,71,2017-05-19


In [21]:
print(df_patient_response.columns, df_patient.columns)

Index(['id', 'checklist_id', 'question_id', 'workday', 'response', 'date',
       'patient_id', 'unit_id', 'user_detail_id', 'hash_fin', 'just_date'],
      dtype='object') Index(['id', 'hash_fin', 'date', 'checklist_id', 'visible_groups', 'los_icu',
       'los', 'apache_is_completed', 'has_antibiotics', 'has_lines',
       'has_medication', 'has_ventilation', 'workday', 'unit_id',
       'user_detail_id', 'just_date'],
      dtype='object')


In [51]:
#matching patients with the answers
df1 = pd.merge(df_patient_response, df_patient[['id', 'hash_fin', 'date', 'checklist_id', 'unit_id',
                                                'visible_groups', 'los_icu', 'user_detail_id', 'just_date']],            
              
               left_on=['hash_fin','user_detail_id','unit_id','checklist_id'], 
               right_on=['hash_fin', 'user_detail_id','unit_id','checklist_id'])
# df_merge_patient_response[['checklist_id_x', 'question_id_x', 'workday_x', 'response', 'date_x', 'patient_id',]]
# df_merge_patient_response.columns

In [88]:
#Please rename this, g1 has all the responses for each patient 
g = df1[df1.date_x >= df1.date_y].groupby(['hash_fin', 'id_x'])#user_detail_id', 'unit_id', 'checklist_id','workday'])#['date_x', 'visible_groups']
g1 = g.apply(lambda x: x.sort_values(by='date_y', ascending=False))
g1.drop_duplicates(subset='id_x', keep='first')

id_x checklist_id  \
hash_fin                                 id_x                          
07e1b75fe53343221703226deb96b71bb6d90930 977  87    977            1   
                                         978  88    978            1   
                                         979  89    979            1   
                                         980  90    980            1   
                                         981  91    981            1   
                                         982  92    982            1   
                                         983  93    983            1   
                                         984  94    984            1   
                                         985  95    985            1   
                                         986  96    986            1   
                                         987  97    987            1   
                                         988  98    988            1   
                                         1139 100  1139            1   
                                         1140 102  1140            1   
                                         1141 104  1141            1   
                                         1142 106  1142            1   
                                         1143 108  1143            1   
                                         1144 110  1144            1   
                                         1145 112  1145            1   
                                         1146 114  1146            1   
                                         1147 116  1147            1   
                                         1150 118  1150            1   
                                         1151 120  1151            1   
2265516f326cf9ea002bfd4a21dc122a63e0e380 2747 490  2747            2   
                                         2748 491  2748            2   
                                         2749 492  2749            2   
                                         2750 493  2750            2   
                                         2751 494  2751            2   
                                         2752 495  2752            2   
                                         2753 496  2753            2   
...                                                 ...          ...   
f3b69dd81ffa2ed50de3166f1f9b99cbb0c6de89 3403 561  3403            2   
                                         3404 562  3404            2   
                                         3406 563  3406            2   
                                         3407 564  3407            2   
                                         3409 565  3409            2   
                                         3410 566  3410            2   
                                         3678 583  3678            2   
                                         3679 584  3679            2   
                                         3680 585  3680            2   
                                         3681 586  3681            2   
                                         3682 587  3682            2   
                                         3683 588  3683            2   
                                         3684 589  3684            2   
                                         3685 590  3685            2   
                                         3923 567  3923            2   
                                         3924 568  3924            2   
                                         3925 569  3925            2   
                                         3926 570  3926            2   
                                         3927 571  3927            2   
                                         3928 572  3928            2   
                                         3929 573  3929            2   
                                         3930 574  3930            2   
                                         4415 575  4415            2   
                   

# Specific Categories

Yeah, so they want to look at three specific questions: 1) DVT, 2) Lines (specifically central lines if their version has it split into peripheral and central), 3) Foleys


[2:01] 
I think for each question, what they want to know is: for each week, 1) how many patients total were the questions answered, 2) for the patients where it was answered, what are the proportions of the different answers, 3) for patients where the 'bad' answer was given initially (no we can't discontinue the foley, no we can't pull the line, etc), how many days did they spend in the 'bad' answer category before switching to the 'good' category. [Taking into account where it was contraindicated]


[2:02] 
they want to see it because they measure all these things by another method (probably nursing reports) and they want to see if those two sources (us vs nursing reports) have any differences

# NSICU (unit 3)
01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 

### CTICU (unit 4)
01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 

In [73]:
checklist_query = "select * from backend_v10_checklist"
df_checklist = pd.read_sql(checklist_query, conn)
df_checklist

,id,public_id,name,text,in_production,unit_id
0,2,20,Test,"{""id"": ""20"", ""groups"": [{""id"": ""01"", ""dependen...",1,1
1,5,1,Initial checklist,"{""order"": ""(01,{03|06|07|08},10)"", ""id"": ""1"", ...",1,3
2,10,1,Initial checklist,"{""order"": ""(01,{03|06|07|08},10)"", ""id"": ""1"", ...",1,4
3,11,4,No ARDS with N/A (lowercase),"{""order"": ""(01,{03|06|07|08},10)"", ""groups"": [...",1,8
4,12,100,Leigh Wild Checklist,"{""order"": ""(01,{03|06|07|08},10)"", ""groups"": [...",1,8
5,13,2,Initial checklist with tracheostomy question,"{""description"": ""Initial checklist with trache...",1,3
6,14,2,Initial checklist with tracheostomy question,"{""description"": ""Initial checklist with trache...",1,4


In [94]:
import ast
unit3_2 =ast.literal_eval(df_checklist.text[2])
unit3_1=ast.literal_eval(df_checklist.text[6])

In [95]:
for key, val in unit3_1.items():
    print(key, val, type(val),len(val))
    print()

id 2 <class 'str'> 1

order (01,{03|06|07|08},10) <class 'str'> 21

groups [{'emrInfoId': 'ventilatorWeaning', 'sections': [{'id': '$0100', 'title': 'Sedation:', 'dependence': 'hasMedication', 'questions': [{'text': 'Has a sedation awakening trial (SAT) been performed today?', 'id': '$0101', 'dependence': 'true', 'indentation': 0, 'questionType': 'Q'}, {'text': 'Is it appropriate to perform an SAT today?', 'id': '$0102', 'dependence': "$0101=='N'", 'indentation': 0, 'questionType': 'A'}]}, {'id': '$0200', 'title': 'Ventilator weaning:', 'dependence': 'hasVentilation', 'questions': [{'text': 'Has a spontaneous breathing trial (SBT) been performed today?', 'id': '$0201', 'dependence': 'true', 'indentation': 0, 'questionType': 'Q'}, {'text': 'Can the patient be extubated today?', 'id': '$0202', 'dependence': "$0201=='Y'", 'indentation': 0, 'questionType': 'Q'}, {'text': 'Does the patient have a tracheostomy or is scheduled for tracheostomy placement?', 'id': '$0205', 'dependence': "$0201=

In [96]:
for a in unit3_1['groups']:
    print(a['id'], a['emrInfoId'], '\n')

01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 



In [97]:
for b in unit3_2['groups']:
    print(b['id'], b['emrInfoId'], '\n')

01 ventilatorWeaning 

03 linesFood 

06 antibiotics 

07 dvt 

08 pressureUlcer 

10 NO 

